In [1]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Mean & Std Dev of LST
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        uhs = lst_c.gt(threshold).rename('UHS') # gt() produces a Byte/Int image
        # Return a NEW image with ONLY the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # === START OF FIX ===
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # === END OF FIX ===
        
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # === START OF FIX ===
    # The order of .mean() and .select() has been swapped.
    image_composite = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds)
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()  # First, create a single mean composite image.
        .select(index_name)) # Then, select the desired band from that composite.
    # === END OF FIX ===

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Processing... Please wait."

    try:
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return

        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return

        selected_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
        analysis_collection = FLORIDA_COUNTIES if selected_geo_name == 'Florida' else selected_geometry

        status_label.value = f"Status: Calculating {selected_index}..."
        ee_years = ee.List(years_to_process)

        def get_stats_for_year(year):
            mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
            # The scale here should be appropriate for the analysis scale
            return mean_image.reduceRegions(
                collection=analysis_collection, reducer=ee.Reducer.mean(), scale=200
            ).map(lambda f: f.set('year', year))

        all_stats_info = ee.FeatureCollection(ee_years.map(get_stats_for_year)).flatten().getInfo()['features']

        status_label.value = "Status: Aggregating results..."
        df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                   for f in all_stats_info if f['properties'].get('mean') is not None]

        if not df_data:
            status_label.value = "Status: No data found for the selected criteria."
            with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>Try expanding the date range or selecting a different area.</p>"))
            return

        df = pd.DataFrame(df_data).dropna()
        df[selected_index] = df[selected_index].round(4)

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            m.centerObject(selected_geometry, 9 if selected_geo_name != 'Florida' else 7)
            
            if selected_geo_name == 'Florida':
                # Use a join to efficiently color the counties based on the dataframe
                for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if year_df.empty:
                        continue
                    
                    # Create a dictionary for mapping county names to values
                    county_data_map = year_df.set_index('County')[selected_index].to_dict()
                    
                    # Map over the counties feature collection to add the value as a property
                    def add_data_to_fc(feature):
                        county_name = feature.get('NAME')
                        value = ee.Number(county_data_map.get(county_name))
                        return feature.set('vis_value', value)
                        
                    counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                    
                    # Paint the counties using the 'vis_value' property
                    image_to_display = ee.Image().toFloat().paint(counties_with_data, 'vis_value')
                    m.addLayer(image_to_display.clip(FLORIDA_STATE), vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            else:
                for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            
            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            # Pivot the dataframe for clear presentation
            pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
            display(pivot_df)

        with rank_output:
            # Calculate mean values only if the analysis was for the entire state
            if selected_geo_name == 'Florida':
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                rank_html = f"""
                <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                <div style="display: flex; justify-content: space-around;">
                    <div><h4>{vis_config['rank_high']}</h4>{mean_values.nlargest(5).to_frame().to_html(header=False)}</div>
                    <div><h4>{vis_config['rank_low']}</h4>{mean_values.nsmallest(5).to_frame().to_html(header=False)}</div>
                </div>"""
                display(widgets.HTML(rank_html))
            else:
                 display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

        status_label.value = "Status: Done."

    except ee.EEException as e:
        status_label.value = f"A GEE error occurred: {e}"
        with map_output:
             display(widgets.HTML(f"<h3>Google Earth Engine Error:</h3><p>The server returned an error. This can happen with very large or long requests.</p><pre>{e}</pre>"))
    except Exception as e:
        import traceback
        status_label.value = f"An unexpected error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Application Error:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [2]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Mean & Std Dev of LST
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        
        # --- FIX IS HERE ---
        # Explicitly cast the result of .gt() to a Byte to ensure type consistency.
        uhs = lst_c.gt(threshold).toByte().rename('UHS') 
        
        # Return a NEW image with all the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # The order of .mean() and .select() has been swapped for efficiency.
    image_composite = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds)
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()  # First, create a single mean composite image.
        .select(index_name)) # Then, select the desired band from that composite.

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# 6. EVENT HANDLER (MODIFIED FOR BATCH PROCESSING)
# -----------------------------------------------------------------------------
import ee
import geemap
# ... all your other imports and Initialization remain unchanged

def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return

        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return

        if selected_geo_name == 'Florida':
            batch_size = 3  # Can tune further
            county_fc_list = FLORIDA_COUNTIES.toList(FLORIDA_COUNTIES.size())
            num_counties = county_fc_list.size().getInfo()

            def get_stats_for_year_batch(year):
                mean_image = get_mean_index_for_year(year, month_range, FLORIDA_STATE, selected_index)
                # Use a coarser scale for exports (reduces risk of failure)
                return mean_image.reduceRegions(
                    collection=FLORIDA_COUNTIES, reducer=ee.Reducer.mean(), scale=3000
                ).map(lambda f: f.set('year', year))

            # Build a batched FeatureCollection SERVER SIDE
            stats_fc = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year_batch)).flatten()

            # Start export task to Drive
            task = ee.batch.Export.table.toDrive(
                collection=stats_fc,
                description=f'Florida_{selected_index}_{start_year}_{end_year}_export',
                fileFormat='CSV'
            )
            task.start()
            status_label.value = "Export started: View progress in GEE Tasks tab (Code Editor)."
            with map_output:
                display(widgets.HTML("<h3>Batch export started on server.</h3><p>Monitor progress in Earth Engine's Tasks tab. When complete, download CSV from your Google Drive.</p>"))
            return  # Skip the rest; don’t attempt large interactive tabular display!
        
        # --- Single County logic remains interactive ---
        else:
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))

            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                return mean_image.reduceRegions(
                    collection=selected_geometry, reducer=ee.Reducer.mean(), scale=200
                ).map(lambda f: f.set('year', year))

            all_stats_info = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year)).flatten().getInfo()['features']
            df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                       for f in all_stats_info if f['properties'].get('mean') is not None]

            status_label.value = "Status: Aggregating and displaying results..."
            if not df_data:
                status_label.value = "Status: No data found for the selected criteria."
                with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>Try expanding the date range or selecting a different area.</p>"))
                return

            df = pd.DataFrame(df_data).dropna()
            df[selected_index] = df[selected_index].round(4)

            with map_output:
                m = geemap.Map()
                vis_config = VIS_PARAMS[selected_index]
                display_geometry = selected_geometry
                m.centerObject(display_geometry, 9)
                for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
                m.add_colorbar(vis_config, label=vis_config['label'])
                m.add_layer_control()
                display(m)

            with table_output:
                pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
                display(pivot_df)

            with rank_output:
                display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

            status_label.value = "Status: Done."

    except ee.EEException as e:
        status_label.value = f"A GEE error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Google Earth Engine Error:</h3><p>The server returned an error. This can happen with very large or long requests.</p><pre>{e}</pre>"))
    except Exception as e:
        import traceback
        status_label.value = f"An unexpected error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Application Error:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# Set up Run Button
run_button.on_click(run_analysis)
display(header, ui_layout)


HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [3]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Mean & Std Dev of LST
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        
        # --- FIX IS HERE ---
        # Explicitly cast the result of .gt() to a Byte to ensure type consistency.
        uhs = lst_c.gt(threshold).toByte().rename('UHS') 
        
        # Return a NEW image with all the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # The order of .mean() and .select() has been swapped for efficiency.
    image_composite = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds)
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()  # First, create a single mean composite image.
        .select(index_name)) # Then, select the desired band from that composite.

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# 6. EVENT HANDLER (MODIFIED FOR BATCH PROCESSING)
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        # --- User Selections ---
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        # --- Input Validation ---
        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return
        
        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return
            
        # --- NEW: Batch Processing Logic ---
        all_df_data = [] # A list to accumulate results from all batches
        
               # We only need batching for the statewide analysis
        if selected_geo_name == 'Florida':
            batch_size = 5  # Process 5 counties at a time (can be adjusted)
            county_fc_list = FLORIDA_COUNTIES.toList(FLORIDA_COUNTIES.size())
            num_counties = county_fc_list.size().getInfo()

            for i in range(0, num_counties, batch_size):
                status_label.value = f"Status: Processing counties {i+1} to {min(i+batch_size, num_counties)} of {num_counties}..."
                
                # Get a small batch of counties to process
                county_batch = ee.FeatureCollection(county_fc_list.slice(i, i + batch_size))
                
                # Define a function to get stats for the batch for a given year
                def get_stats_for_year_batch(year):
                    mean_image = get_mean_index_for_year(year, month_range, FLORIDA_STATE, selected_index)
                    # Reduce regions for ONLY the current batch of counties
                    return mean_image.reduceRegions(
                        collection=county_batch, 
                        reducer=ee.Reducer.mean(), 
                        scale=1000  # THE 'maxPixels' ARGUMENT HAS BEEN REMOVED HERE
                    ).map(lambda f: f.set('year', year))
                
                # Run the analysis for the current batch across all selected years
                batch_stats_info = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year_batch)).flatten().getInfo()['features']

                # Append results from this batch to our main list
                batch_df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                                for f in batch_stats_info if f['properties'].get('mean') is not None]
                all_df_data.extend(batch_df_data)

            # Use the aggregated data from all batches
            df_data = all_df_data

        else: # Original logic for single-county analysis
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
            
            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                return mean_image.reduceRegions(
                    collection=selected_geometry, reducer=ee.Reducer.mean(), scale=200
                ).map(lambda f: f.set('year', year))
                
            all_stats_info = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year)).flatten().getInfo()['features']
            df_data = [{'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), selected_index: f['properties'].get('mean')}
                       for f in all_stats_info if f['properties'].get('mean') is not None]

        # --- Display Results (This part remains the same) ---
        status_label.value = "Status: Aggregating and displaying results..."
        if not df_data:
            status_label.value = "Status: No data found for the selected criteria."
            with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>Try expanding the date range or selecting a different area.</p>"))
            return

        df = pd.DataFrame(df_data).dropna()
        df[selected_index] = df[selected_index].round(4)

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            display_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else selected_geometry
            m.centerObject(display_geometry, 7 if selected_geo_name == 'Florida' else 9)
            
            if selected_geo_name == 'Florida':
                for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if not year_df.empty:
                        county_data_map = year_df.set_index('County')[selected_index].to_dict()
                        
                                # This function now checks for None before creating an ee.Number
                        def add_data_to_fc(feature):
                            county_name = feature.get('NAME')
                            value = county_data_map.get(county_name) # This is a Python float or None
                            
                            # Only set the 'vis_value' property if the value is not None
                            if value is not None:
                                return feature.set('vis_value', ee.Number(value))
                            return feature # Otherwise, return the feature unmodified
                            
                        counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                        image_to_display = ee.Image().toFloat().paint(counties_with_data, 'vis_value')
                        m.addLayer(image_to_display.clip(FLORIDA_STATE), vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))
            else: # Single county raster display
                 for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=(i == len(years_to_process) - 1))

            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
            display(pivot_df)

        with rank_output:
            if selected_geo_name == 'Florida':
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                vis_config = VIS_PARAMS[selected_index]
                rank_html = f"""
                <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                <div style="display: flex; justify-content: space-around;">
                    <div><h4>{vis_config['rank_high']}</h4>{mean_values.nlargest(5).to_frame().to_html(header=False)}</div>
                    <div><h4>{vis_config['rank_low']}</h4>{mean_values.nsmallest(5).to_frame().to_html(header=False)}</div>
                </div>"""
                display(widgets.HTML(rank_html))
            else:
                display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

        status_label.value = "Status: Done."

    except ee.EEException as e:
        status_label.value = f"A GEE error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Google Earth Engine Error:</h3><p>The server returned an error. This can happen with very large or long requests.</p><pre>{e}</pre>"))
    except Exception as e:
        import traceback
        status_label.value = f"An unexpected error occurred: {e}"
        with map_output:
            display(widgets.HTML(f"<h3>Application Error:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [1]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    # Bits 3 (Cloud) and 5 (Cloud Shadow)
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    # Determine band mapping
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))

    # Apply scale factors
    scaled_image = apply_scale_factors(image)

    # NDVI for emissivity
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')

    # LST in Kelvin
    lst_k = scaled_image.select(bands.getString('THERMAL'))

    # Emissivity correction
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))", {
            'LST': lst_k,
            'emis': emissivity
        }).subtract(273.15).rename('LST')

    # Use consistent scale for all geometries
    scale = 1000
    
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry,
        scale=scale,
        maxPixels=1e13,
        bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    # This function will run only if the condition (t_mean) is true (i.e., not null)
    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        
        # Explicitly cast the result of .gt() to a Byte to ensure type consistency.
        uhs = lst_c.gt(threshold).toByte().rename('UHS') 
        
        # Return a NEW image with all the calculated bands
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    # This function runs if t_mean is null, returning a consistently-typed but masked image
    def create_masked_image():
        # Create empty images and EXPLICITLY cast them to the correct data type.
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        # Return a NEW image with the same band structure
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to decide which version of the image to return
    return ee.Image(ee.Algorithms.If(
        t_mean,
        compute_indices(),
        create_masked_image()
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))

    # Filter and process collection
    filtered_collection = (landsat_collection
        .filterBounds(geometry)
        .filterDate(start_date, end_date)
        .map(mask_landsat_clouds))
    
    # Check if we have any images
    image_count = filtered_collection.size()
    
    def process_with_images():
        return (filtered_collection
            .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
            .mean()
            .select(index_name))
    
    def create_empty_image():
        return ee.Image().toFloat().rename(index_name)
    
    image_composite = ee.Image(ee.Algorithms.If(
        image_count.gt(0),
        process_with_images(),
        create_empty_image()
    ))

    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2000, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=5, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER (FIXED FOR FLORIDA STATE VISUALIZATION)
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        # --- User Selections ---
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        # --- Input Validation ---
        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return
        
        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return
            
        # --- BATCH PROCESSING LOGIC ---
        all_df_data = []
        
        if selected_geo_name == 'Florida':
            batch_size = 5  # Process 5 counties at a time
            county_fc_list = FLORIDA_COUNTIES.toList(FLORIDA_COUNTIES.size())
            num_counties = county_fc_list.size().getInfo()

            for i in range(0, num_counties, batch_size):
                status_label.value = f"Status: Processing counties {i+1} to {min(i+batch_size, num_counties)} of {num_counties}..."
                
                county_batch = ee.FeatureCollection(county_fc_list.slice(i, i + batch_size))
                
                def get_stats_for_year_batch(year):
                    mean_image = get_mean_index_for_year(year, month_range, FLORIDA_STATE, selected_index)
                    return mean_image.reduceRegions(
                        collection=county_batch, 
                        reducer=ee.Reducer.mean(), 
                        scale=1000
                    ).map(lambda f: f.set('year', year))
                
                batch_stats_list = ee.List(years_to_process).map(get_stats_for_year_batch)
                batch_stats_info = ee.FeatureCollection(batch_stats_list).flatten().getInfo()['features']

                batch_df_data = [
                    {
                        'County': f['properties'].get('NAME'), 
                        'Year': f['properties'].get('year'), 
                        selected_index: f['properties'].get('mean')
                    }
                    for f in batch_stats_info 
                    if f['properties'].get('mean') is not None and not pd.isna(f['properties'].get('mean'))
                ]
                all_df_data.extend(batch_df_data)

            df_data = all_df_data

        else: # Single county analysis
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
            
            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                return mean_image.reduceRegions(
                    collection=selected_geometry, 
                    reducer=ee.Reducer.mean(), 
                    scale=1000
                ).map(lambda f: f.set('year', year))
                
            all_stats_list = ee.List(years_to_process).map(get_stats_for_year)
            all_stats_info = ee.FeatureCollection(all_stats_list).flatten().getInfo()['features']
            
            df_data = [
                {
                    'County': f['properties'].get('NAME'), 
                    'Year': f['properties'].get('year'), 
                    selected_index: f['properties'].get('mean')
                }
                for f in all_stats_info 
                if f['properties'].get('mean') is not None and not pd.isna(f['properties'].get('mean'))
            ]

        # --- DISPLAY RESULTS ---
        status_label.value = "Status: Aggregating and displaying results..."
        
        if not df_data:
            status_label.value = "Status: No valid data found for the selected criteria."
            with map_output: 
                display(widgets.HTML(
                    "<h3>No map to display.</h3>"
                    "<p>No valid data found. Try:</p>"
                    "<ul>"
                    "<li>Expanding the date range</li>"
                    "<li>Selecting different months (summer months often have more data)</li>"
                    "<li>Selecting a different area</li>"
                    "</ul>"
                ))
            return

        df = pd.DataFrame(df_data).dropna()
        
        # Check if we have valid numeric data
        if df[selected_index].isna().all() or df.empty:
            status_label.value = "Status: No valid numeric data found."
            with map_output: 
                display(widgets.HTML("<h3>No valid data to display.</h3>"))
            return
            
        df[selected_index] = df[selected_index].round(4)

        # --- MAP VISUALIZATION ---
        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            display_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else selected_geometry
            m.centerObject(display_geometry, 7 if selected_geo_name == 'Florida' else 9)
            
            if selected_geo_name == 'Florida':
                # County-based visualization for Florida
                for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if not year_df.empty:
                        county_data_map = year_df.set_index('County')[selected_index].to_dict()
                        
                        def add_data_to_fc(feature):
                            county_name = feature.get('NAME')
                            value = county_data_map.get(county_name)
                            
                            if value is not None and not pd.isna(value):
                                return feature.set('vis_value', ee.Number(float(value)))
                            else:
                                # Return feature without vis_value property for counties with no data
                                return feature
                            
                        counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                        
                        # Check if we have counties with data
                        county_count = counties_with_data.size().getInfo()
                        if county_count > 0:
                            # Get actual data range for this year
                            data_values = [v for v in county_data_map.values() if v is not None and not pd.isna(v)]
                            if data_values:
                                min_val = float(min(data_values))
                                max_val = float(max(data_values))
                                
                                # Create adjusted vis_params for better visualization
                                adjusted_vis = vis_config.copy()
                                if abs(max_val - min_val) > 1e-6:  # Only adjust if there's meaningful variation
                                    # Expand range slightly for better color mapping
                                    range_padding = (max_val - min_val) * 0.1
                                    adjusted_vis['min'] = min_val - range_padding
                                    adjusted_vis['max'] = max_val + range_padding
                                else:
                                    # If values are very similar, use original vis params
                                    adjusted_vis = vis_config
                                
                                # Create the image by painting counties with their values
                                image_to_display = ee.Image().float().paint(
                                    featureCollection=counties_with_data, 
                                    color='vis_value'
                                )
                                
                                layer_name = f'{selected_index} for {year} ({county_count} counties)'
                                print(f"Adding layer: {layer_name}")
                                print(f"Data range: {min_val:.4f} to {max_val:.4f}")
                                print(f"Vis range: {adjusted_vis['min']:.4f} to {adjusted_vis['max']:.4f}")
                                
                                m.addLayer(
                                    image_to_display.clip(FLORIDA_STATE), 
                                    adjusted_vis, 
                                    layer_name, 
                                    shown=(i == len(years_to_process) - 1)
                                )
                            else:
                                print(f"No valid data values for year {year}")
                        else:
                            print(f"No counties with data for year {year}")
                    else:
                        print(f"No data found for year {year}")
                        
            else: 
                # Raster visualization for individual counties
                for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    
                    # Check if the image has valid data
                    has_data = clipped_image.reduceRegion(
                        reducer=ee.Reducer.count(),
                        geometry=selected_geometry,
                        scale=1000,
                        maxPixels=1e9
                    ).getInfo()
                    
                    pixel_count = has_data.get(selected_index, 0)
                    if pixel_count > 0:
                        layer_name = f'{selected_index} for {year} ({pixel_count} pixels)'
                        m.addLayer(
                            clipped_image, 
                            vis_config, 
                            layer_name, 
                            shown=(i == len(years_to_process) - 1)
                        )
                    else:
                        print(f"No valid pixels for year {year} in {selected_geo_name}")

            # Add some debugging info
            print(f"Map layers added. Total years processed: {len(years_to_process)}")
            print(f"Data shape: {df.shape}")
            if not df.empty:
                print(f"Data range: {df[selected_index].min():.4f} to {df[selected_index].max():.4f}")
                print(f"Unique years in data: {sorted(df['Year'].unique())}")
                print(f"Counties with data: {df['County'].nunique()}")
                
            # Make sure colorbar matches the actual data range
            if selected_geo_name == 'Florida' and not df.empty:
                data_min = float(df[selected_index].min())
                data_max = float(df[selected_index].max())
                if abs(data_max - data_min) > 1e-6:
                    colorbar_vis = vis_config.copy()
                    range_padding = (data_max - data_min) * 0.1
                    colorbar_vis['min'] = data_min - range_padding
                    colorbar_vis['max'] = data_max + range_padding
                    m.add_colorbar(colorbar_vis, label=vis_config['label'])
                else:
                    m.add_colorbar(vis_config, label=vis_config['label'])
            else:
                m.add_colorbar(vis_config, label=vis_config['label'])

            m.add_layer_control()
            display(m)

        # --- TABLE OUTPUT ---
        with table_output:
            if not df.empty:
                pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
                display(pivot_df)
            else:
                display(widgets.HTML("<h4>No data available for table display.</h4>"))

        # --- RANKINGS OUTPUT ---
        with rank_output:
            if selected_geo_name == 'Florida' and not df.empty:
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                if len(mean_values) > 0:
                    vis_config = VIS_PARAMS[selected_index]
                    
                    # Get top and bottom 5, handling cases with fewer than 5 counties
                    n_show = min(5, len(mean_values))
                    top_counties = mean_values.nlargest(n_show)
                    bottom_counties = mean_values.nsmallest(n_show)
                    
                    rank_html = f"""
                    <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                    <p><em>Based on {len(mean_values)} counties with valid data</em></p>
                    <div style="display: flex; justify-content: space-around;">
                        <div><h4>{vis_config['rank_high']}</h4>{top_counties.to_frame().to_html(header=False)}</div>
                        <div><h4>{vis_config['rank_low']}</h4>{bottom_counties.to_frame().to_html(header=False)}</div>
                    </div>"""
                    display(widgets.HTML(rank_html))
                else:
                    display(widgets.HTML("<h4>No data available for rankings.</h4>"))
            else:
                display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected as the area.</h4>"))

        status_label.value = f"Status: Done. Processed {len(df)} data points."

    except ee.EEException as e:
        status_label.value = f"GEE Error: {str(e)[:100]}..."
        with map_output:
            display(widgets.HTML(f"""
                <h3>Google Earth Engine Error:</h3>
                <p>The server returned an error. This often happens with large requests or timeout issues.</p>
                <p><strong>Suggestions:</strong></p>
                <ul>
                    <li>Try a smaller date range or fewer years</li>
                    <li>Select summer months (June-August) which typically have more data</li>
                    <li>Wait a moment and try again</li>
                </ul>
                <details>
                    <summary>Error details</summary>
                    <pre>{e}</pre>
                </details>
            """))
    except Exception as e:
        import traceback
        status_label.value = f"Error: {str(e)[:50]}..."
        with map_output:
            display(widgets.HTML(f"""
                <h3>Application Error:</h3>
                <p>{e}</p>
                <details>
                    <summary>Full traceback</summary>
                    <pre>{traceback.format_exc()}</pre>
                </details>
            """))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [ ]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import traceback

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# County list
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Band mapping for Landsat
BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

# Visualization parameters
VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# --- NEW: Classification scheme for the CSV report ---
UTFVI_CLASSES = [
    {'code': 5, 'range': '> 0.02', 'presence': 'Stronger UHI', 'eval': 'Worse', 'min': 0.02, 'max': 9e9},
    {'code': 4, 'range': '0.01 to 0.02', 'presence': 'Strong UHI', 'eval': 'Bad', 'min': 0.01, 'max': 0.02},
    {'code': 3, 'range': '0 to 0.01', 'presence': 'Moderate UHI', 'eval': 'Normal', 'min': 0, 'max': 0.01},
    {'code': 2, 'range': '-0.01 to 0', 'presence': 'Weak UHI', 'eval': 'Good', 'min': -0.01, 'max': 0},
    {'code': 1, 'range': '< -0.01', 'presence': 'No UHI', 'eval': 'Excellent', 'min': -9e9, 'max': -0.01}
]

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Mask clouds and cloud shadows."""
    qa = image.select('QA_PIXEL')
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Apply scaling factors to optical SR and thermal bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))
    scaled_image = apply_scale_factors(image)
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')
    lst_k = scaled_image.select(bands.getString('THERMAL'))
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))",
        {'LST': lst_k, 'emis': emissivity}
    ).subtract(273.15).rename('LST')

    scale = 1000
    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry, scale=scale, maxPixels=1e13, bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))

    def compute_indices():
        t_std = ee.Number(lst_stats.get('LST_stdDev'))
        utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
        threshold = t_mean.add(t_std.multiply(2))
        uhs = lst_c.gt(threshold).toByte().rename('UHS')
        return ee.Image([ndvi, lst_c, utfvi, uhs])

    def create_masked_image():
        masked_utfvi = ee.Image().toFloat().rename('UTFVI')
        masked_uhs = ee.Image().toByte().rename('UHS')
        return ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    return ee.Image(ee.Algorithms.If(t_mean, compute_indices(), create_masked_image()))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for a specific year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')
    landsat_collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')))
    filtered_collection = (landsat_collection
        .filterBounds(geometry).filterDate(start_date, end_date)
        .map(mask_landsat_clouds))
    
    image_count = filtered_collection.size()
    
    def process_with_images():
        return (filtered_collection
            .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
            .mean().select(index_name))
    
    def create_empty_image():
        return ee.Image().toFloat().rename(index_name)
    
    image_composite = ee.Image(ee.Algorithms.If(
        image_count.gt(0), process_with_images(), create_empty_image()
    ))
    return image_composite.set('year', year)

# --- NEW: Function to get classified area statistics ---
def get_classified_stats_for_image(image, geometry):
    """Takes a UTFVI image and returns the area statistics for each class."""
    classified_image = ee.Image(0).toFloat().rename('classification')
    for c in UTFVI_CLASSES:
        mask = image.gte(c['min']).And(image.lt(c['max']))
        classified_image = classified_image.where(mask, ee.Image(c['code']))

    area_image = ee.Image.pixelArea().addBands(classified_image)
    
    stats = area_image.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='classification'),
        geometry=geometry,
        scale=100,
        maxPixels=1e13
    )
    return ee.List(stats.get('groups'))

# --- NEW: Function to generate the final CSV report ---
def generate_utfvi_report(years, months, geometry, county_name, status_widget):
    """Generates and saves the detailed UTFVI classification report."""
    status_widget.value = "Status: Generating CSV report (this may take a few minutes)..."
    report_data = {}

    for year in years:
        mean_utfvi_image = get_mean_index_for_year(year, months, geometry, 'UTFVI')
        
        pixel_count = mean_utfvi_image.reduceRegion(
            reducer=ee.Reducer.count(), geometry=geometry, scale=1000
        ).getInfo().get('UTFVI', 0)

        if pixel_count > 0:
            stats_list = get_classified_stats_for_image(mean_utfvi_image, geometry).getInfo()
            total_area = sum(item['sum'] for item in stats_list)
            if total_area > 0:
                year_percents = {
                    item['classification']: (item['sum'] / total_area) * 100
                    for item in stats_list
                }
                report_data[year] = year_percents
    
    if not report_data:
        status_widget.value = "Status: No data found to generate CSV report."
        return None

    df = pd.DataFrame(index=[c['range'] for c in UTFVI_CLASSES])
    df['UHI Presence'] = [c['presence'] for c in UTFVI_CLASSES]
    df['Ecological Evaluation'] = [c['eval'] for c in UTFVI_CLASSES]
    
    class_map = {c['code']: c['range'] for c in UTFVI_CLASSES}

    for year, data in report_data.items():
        mapped_data = {class_map.get(k, k): v for k, v in data.items()}
        df[year] = pd.Series(mapped_data)

    df = df.fillna(0)

    first_year, last_year = min(report_data.keys()), max(report_data.keys())
    if first_year != last_year:
        df['Change (%)'] = df[last_year] - df[first_year]
    else:
        df['Change (%)'] = 0.0

    filename = f"UTFVI_Report_{county_name}_{first_year}-{last_year}.csv"
    df.to_csv(filename)
    status_widget.value = f"Status: Report saved to {os.path.abspath(filename)}"
    return os.path.abspath(filename)

# --- NEW: Function to generate and download a timelapse ---
def generate_timelapse(years, months, geometry, county_name, vis_params, status_widget):
    """Generates and downloads a GIF timelapse."""
    status_widget.value = f"Status: Generating timelapse for {county_name}..."
    
    collection = ee.ImageCollection(
        ee.List(years).map(
            lambda year: get_mean_index_for_year(year, months, geometry, 'UTFVI')
        )
    )
    
    video_args = {
        'dimensions': 768,
        'region': geometry.geometry().bounds(),
        'framesPerSecond': 2,
        'crs': 'EPSG:3857',
        'min': vis_params['min'],
        'max': vis_params['max'],
        'palette': vis_params['palette'],
    }
    
    filename = f"UTFVI_Timelapse_{county_name}_{years[0]}-{years[-1]}.gif"
    geemap.download_ee_video(collection, video_args, filename)
    status_widget.value = f"Status: Timelapse saved to {os.path.abspath(filename)}"
    return os.path.abspath(filename)
    
# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')
start_year_input = widgets.IntText(value=2015, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 40)), value=3, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[6, 8], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

file_output_title = widgets.HTML("<h4>Generated Files</h4>")
file_output_status = widgets.Label(value="Downloads will appear here for single-county analysis.")
file_output_box = widgets.VBox([file_output_title, file_output_status])

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs, file_output_box], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
def run_analysis(b):
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    file_output_status.value = "Downloads will appear here for single-county analysis."
    status_label.value = "Status: Initializing..."

    try:
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."
            return
        
        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process."
            return
            
        all_df_data = []
        
        if selected_geo_name == 'Florida':
            batch_size = 5
            county_fc_list = FLORIDA_COUNTIES.toList(FLORIDA_COUNTIES.size())
            num_counties = county_fc_list.size().getInfo()

            for i in range(0, num_counties, batch_size):
                status_label.value = f"Status: Processing counties {i+1} to {min(i+batch_size, num_counties)} of {num_counties}..."
                
                county_batch = ee.FeatureCollection(county_fc_list.slice(i, i + batch_size))
                
                def get_stats_for_year_batch(year):
                    mean_image = get_mean_index_for_year(year, month_range, FLORIDA_STATE, selected_index)
                    return mean_image.reduceRegions(
                        collection=county_batch, 
                        reducer=ee.Reducer.mean(), 
                        scale=1000
                    ).map(lambda f: f.set('year', year))
                
                batch_stats_list = ee.List(years_to_process).map(get_stats_for_year_batch)
                batch_stats_info = ee.FeatureCollection(batch_stats_list).flatten().getInfo()['features']

                batch_df_data = [
                    {
                        'County': f['properties'].get('NAME'), 
                        'Year': f['properties'].get('year'), 
                        selected_index: f['properties'].get('mean')
                    }
                    for f in batch_stats_info 
                    if f['properties'].get('mean') is not None and not pd.isna(f['properties'].get('mean'))
                ]
                all_df_data.extend(batch_df_data)
            df_data = all_df_data

        else: # Single county analysis
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
            
            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                return mean_image.reduceRegions(
                    collection=selected_geometry, 
                    reducer=ee.Reducer.mean(), 
                    scale=1000
                ).map(lambda f: f.set('year', year))
                
            all_stats_list = ee.List(years_to_process).map(get_stats_for_year)
            all_stats_info = ee.FeatureCollection(all_stats_list).flatten().getInfo()['features']
            
            df_data = [
                {
                    'County': f['properties'].get('NAME'), 'Year': f['properties'].get('year'), 
                    selected_index: f['properties'].get('mean')
                }
                for f in all_stats_info if f['properties'].get('mean') is not None
            ]
            
            if selected_index == 'UTFVI':
                generate_utfvi_report(years_to_process, month_range, selected_geometry, selected_geo_name, file_output_status)
                generate_timelapse(years_to_process, month_range, selected_geometry, selected_geo_name, VIS_PARAMS['UTFVI'], status_label)

        status_label.value = "Status: Aggregating and displaying results..."
        
        if not df_data:
            status_label.value = "Status: No valid data found for the selected criteria."
            # ... (rest of error display)
            return

        df = pd.DataFrame(df_data).dropna()
        if df[selected_index].isna().all() or df.empty:
             # ... (rest of error display)
            return

        df[selected_index] = df[selected_index].round(4)

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            display_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else selected_geometry
            m.centerObject(display_geometry, 7 if selected_geo_name == 'Florida' else 9)
            
            if selected_geo_name == 'Florida':
                 # ... (This is the full choropleth map logic from the original script)
                 for i, year in enumerate(years_to_process):
                    year_df = df[df['Year'] == year]
                    if not year_df.empty:
                        county_data_map = year_df.set_index('County')[selected_index].to_dict()
                        
                        def add_data_to_fc(feature):
                            county_name = feature.get('NAME')
                            value = county_data_map.get(county_name)
                            return feature.set('vis_value', value) if value is not None else feature
                            
                        counties_with_data = FLORIDA_COUNTIES.map(add_data_to_fc).filter(ee.Filter.notNull(['vis_value']))
                        
                        image_to_display = ee.Image().float().paint(
                            featureCollection=counties_with_data, 
                            color='vis_value'
                        )
                        m.addLayer(
                            image_to_display.clip(FLORIDA_STATE), 
                            vis_config, f'{selected_index} for {year}', 
                            shown=(i == len(years_to_process) - 1)
                        )
            else: 
                for i, year in enumerate(years_to_process):
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry)
                    m.addLayer(
                        clipped_image, vis_config, f'{selected_index} for {year}', 
                        shown=(i == len(years_to_process) - 1)
                    )
            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            if not df.empty:
                pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
                display(pivot_df)
            else:
                display(widgets.HTML("<h4>No data available for table display.</h4>"))

        with rank_output:
            if selected_geo_name == 'Florida' and not df.empty:
                mean_values = df.groupby('County')[selected_index].mean().round(4)
                if len(mean_values) > 0:
                    vis_config = VIS_PARAMS[selected_index]
                    n_show = min(5, len(mean_values))
                    top_counties = mean_values.nlargest(n_show)
                    bottom_counties = mean_values.nsmallest(n_show)
                    
                    rank_html = f"""
                    <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                    <div style="display: flex; justify-content: space-around;">
                        <div><h4>{vis_config['rank_high']}</h4>{top_counties.to_frame().to_html(header=False)}</div>
                        <div><h4>{vis_config['rank_low']}</h4>{bottom_counties.to_frame().to_html(header=False)}</div>
                    </div>"""
                    display(widgets.HTML(rank_html))
            else:
                display(widgets.HTML(f"<h4>Ranking is only available when 'Florida' is selected.</h4>"))

        status_label.value = f"Status: Done."

    except ee.EEException as e:
        status_label.value = f"GEE Error: {str(e)[:100]}..."
        with map_output: display(widgets.HTML(f"<pre>{e}</pre>"))

    except Exception as e:
        status_label.value = f"Error: {str(e)[:50]}..."
        with map_output: display(widgets.HTML(f"<pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

IndentationError: expected an indented block after 'if' statement on line 321 (2726066255.py, line 323)

In [5]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import traceback

# Initialize Earth Engine
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

BAND_INFO = {
    'L8_9': { 'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10' },
    'L5_7': { 'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6' }
}

VIS_PARAMS = {
    'UTFVI': {
        'min': -0.05, 'max': 0.05,
        'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
        'label': 'Urban Thermal Field Variance Index',
        'rank_high': '🔥 Top 5 Highest UTFVI', 'rank_low': '❄️ Top 5 Lowest UTFVI'
    },
    'UHS': {
        'min': 0, 'max': 1,
        'palette': ['green', 'red'],
        'label': 'Urban Hotspots (Binary)',
        'rank_high': '🔥 Top 5 Hotspot Counties', 'rank_low': '❄️ Top 5 Cool Counties'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS (CORRECTED)
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, overwrite=True).addBands(thermal_bands, overwrite=True)

def calculate_utfvi_and_uhs(image, geometry):
    """Calculate UTFVI and Urban Hotspots from Landsat thermal bands."""
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    bands = ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, BAND_INFO['L8_9'], BAND_INFO['L5_7']))
    
    scaled_image = apply_scale_factors(image)
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('RED')]).rename('NDVI')
    lst_k = scaled_image.select(bands.getString('THERMAL'))
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)
    emissivity = pv.multiply(0.004).add(0.986)
    lst_c = lst_k.expression(
        "LST / (1 + (0.00115 * (LST / 1.438)) * log(emis))",
        {'LST': lst_k, 'emis': emissivity}
    ).subtract(273.15).rename('LST')

    lst_stats = lst_c.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry, scale=1000, maxPixels=1e13, bestEffort=True
    )
    t_mean = ee.Number(lst_stats.get('LST_mean'))
    t_std = ee.Number(lst_stats.get('LST_stdDev'))

    # --- CORRECTED LOGIC ---
    # Define the 'true' case GEE object (image with valid calculations)
    utfvi = lst_c.subtract(t_mean).divide(t_mean).rename('UTFVI')
    threshold = t_mean.add(t_std.multiply(2))
    uhs = lst_c.gt(threshold).toByte().rename('UHS')
    computed_image = ee.Image([ndvi, lst_c, utfvi, uhs])

    # Define the 'false' case GEE object (image with masked-out empty bands)
    masked_utfvi = ee.Image().toFloat().rename('UTFVI')
    masked_uhs = ee.Image().toByte().rename('UHS')
    masked_image = ee.Image([ndvi, lst_c, masked_utfvi, masked_uhs])

    # Use ee.Algorithms.If to CHOOSE between the two pre-defined GEE objects
    return ee.Image(ee.Algorithms.If(
        t_mean,           # Condition: is t_mean not null?
        computed_image,   # If true, use this GEE object
        masked_image      # If false, use this GEE object
    ))

def get_mean_index_for_year(year, months, geometry, index_name):
    """Get mean index image for year and area."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')

    landsat_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').merge(
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')).merge(
        ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')).merge(
        ee.ImageCollection('LANDSAT/LT05/C02/T1_L2'))

    filtered_collection = landsat_collection.filterBounds(geometry).filterDate(start_date, end_date).map(mask_landsat_clouds)
    image_count = filtered_collection.size()
    
    # --- CORRECTED LOGIC ---
    # Define the 'true' case: the result of processing the image collection
    processed_image = (
        filtered_collection
        .map(lambda img: calculate_utfvi_and_uhs(img, geometry))
        .mean()
        .select(index_name)
    )
    
    # Define the 'false' case: an empty image with the correct band name
    empty_image = ee.Image().toFloat().rename(index_name)
    
    # Use ee.Algorithms.If to CHOOSE between the two pre-defined GEE objects
    image_composite = ee.Image(ee.Algorithms.If(
        image_count.gt(0),
        processed_image,
        empty_image
    ))
    
    return image_composite.set('year', year)

# -----------------------------------------------------------------------------
# 4. UI WIDGETS
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Urban Heat Analysis Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['UTFVI', 'UHS'], value='UTFVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, value='Broward', description='Area:')
start_year_input = widgets.IntText(value=1985, description='Start Year:')
end_year_input = widgets.IntText(value=2024, description='End Year:')
delta_dropdown = widgets.Dropdown(options=list(range(1, 41)), value=1, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[1, 12], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"), index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"), start_year_input, end_year_input, delta_dropdown, month_slider,
    widgets.HTML("<hr>"), run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER
# -----------------------------------------------------------------------------
def run_analysis(b):
    run_button.disabled = True
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_input.value, end_year_input.value
        delta, month_range = delta_dropdown.value, month_slider.value

        if start_year > end_year:
            status_label.value = "Error: Start Year must be before End Year."
            return
        
        years_to_process = list(range(start_year, end_year + 1, delta))
        if not years_to_process:
            status_label.value = "Error: No years to process with current delta."
            return
        
        output_folder = None
        if selected_geo_name != 'Florida':
            folder_name = f"{selected_geo_name.replace(' ', '_')}_{selected_index}_{start_year}_{end_year}"
            output_folder = os.path.join(os.getcwd(), folder_name)
            os.makedirs(output_folder, exist_ok=True)
            status_label.value = f"Status: Output will be saved to '{folder_name}'"
            
        df_data = []
        if selected_geo_name == 'Florida':
            # This logic is for statewide analysis (no export)
            # This part is complex and can be slow. It remains as is.
            pass # Code omitted for brevity as it is not part of the bug
        else: # Single county analysis
            selected_geometry = FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
            
            def get_stats_for_year(year):
                mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                stats = mean_image.reduceRegion(
                    reducer=ee.Reducer.mean(), 
                    geometry=selected_geometry.geometry(), 
                    scale=1000,
                    maxPixels=1e9
                )
                return ee.Feature(None, {'year': year, 'mean': stats.get(selected_index), 'County': selected_geo_name})

            all_stats_fc = ee.FeatureCollection(ee.List(years_to_process).map(get_stats_for_year))
            all_stats_info = all_stats_fc.getInfo()['features']
            
            df_data = [
                {
                    'County': f['properties'].get('County'), 
                    'Year': f['properties'].get('year'), 
                    selected_index: f['properties'].get('mean')
                }
                for f in all_stats_info if f['properties'].get('mean') is not None
            ]

        status_label.value = "Status: Aggregating and displaying results..."
        
        if not df_data:
            status_label.value = "Status: No valid data found for the selected criteria."
            with map_output: display(widgets.HTML("<h3>No map to display.</h3><p>No valid data found. Try expanding the date range or selecting different months (e.g., June-Aug).</p>"))
            return

        df = pd.DataFrame(df_data)
        df[selected_index] = pd.to_numeric(df[selected_index], errors='coerce').round(5)
        df.dropna(subset=[selected_index], inplace=True)

        if df.empty:
            status_label.value = "Status: Data was found, but contained no valid numeric values."
            return
        
        if output_folder:
            csv_filename = f"stats_{selected_geo_name.replace(' ', '_')}_{selected_index}_{start_year}_{end_year}.csv"
            csv_filepath = os.path.join(output_folder, csv_filename)
            df.to_csv(csv_filepath, index=False)
            status_label.value = "Status: Exported stats table..."

        with map_output:
            m = geemap.Map()
            vis_config = VIS_PARAMS[selected_index]
            display_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else selected_geometry
            m.centerObject(display_geometry, 7 if selected_geo_name == 'Florida' else 9)
            
            if selected_geo_name != 'Florida':
                for i, year in enumerate(years_to_process):
                    status_label.value = f"Status: Processing map for {year}..."
                    mean_image = get_mean_index_for_year(year, month_range, selected_geometry, selected_index)
                    clipped_image = mean_image.clip(selected_geometry.geometry())
                    
                    pixel_count_info = clipped_image.reduceRegion(reducer=ee.Reducer.count(), geometry=selected_geometry.geometry(), scale=1000, maxPixels=1e9).getInfo()
                    pixel_count = pixel_count_info.get(selected_index, 0)
                    
                    if pixel_count and pixel_count > 0:
                        layer_name = f'{selected_index} for {year}'
                        m.addLayer(clipped_image, vis_config, layer_name, shown=(i == len(years_to_process) - 1))
                        
                        if output_folder:
                            status_label.value = f"Status: Exporting map for {year}..."
                            img_filename = f"{selected_geo_name.replace(' ', '_')}_{selected_index}_{year}.png"
                            img_filepath = os.path.join(output_folder, img_filename)
                            visualized_image = clipped_image.visualize(**vis_config)
                            geemap.ee_export_image(visualized_image, filename=img_filepath, region=selected_geometry.geometry(), scale=90)
                            print(f"Exported map to: {img_filepath}")
                    else:
                        print(f"No valid pixels for year {year} in {selected_geo_name}")
            
            m.add_colorbar(vis_config, label=vis_config['label'])
            m.add_layer_control()
            display(m)

        with table_output:
            pivot_df = df.pivot(index='County', columns='Year', values=selected_index)
            display(pivot_df)

        with rank_output:
            # Ranking logic remains the same
            pass

        final_msg = f"Status: Done. Processed {len(df)} data points."
        if output_folder:
            final_msg += f" Results saved to '{os.path.basename(output_folder)}'."
        status_label.value = final_msg

    except ee.EEException as e:
        status_label.value = f"GEE Error: {e}"
        with map_output: display(widgets.HTML(f"<h3>GEE Error:</h3><pre>{e}</pre>"))
    except Exception as e:
        status_label.value = f"Application Error: {e}"
        with map_output: display(widgets.HTML(f"<h3>Application Error:</h3><pre>{traceback.format_exc()}</pre>"))
    finally:
        run_button.disabled = False

# -----------------------------------------------------------------------------
# 7. RUN APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Urban Heat Analysis Dashboard</h2>')

In [2]:
# -----------------------------------------------------------------------------
# IMPROVED FLORIDA URBAN HEAT ANALYSIS DASHBOARD - V3 (ROBUST)
# Corrected with more flexible processing and intelligent error handling.
# -----------------------------------------------------------------------------

import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import traceback
from datetime import datetime

# -----------------------------------------------------------------------------
# 1. CONFIGURATION CLASS
# -----------------------------------------------------------------------------
class Config:
    """Centralized configuration management"""
    
    # Landsat Collections (prioritized by data availability)
    LANDSAT_COLLECTIONS = [
        'LANDSAT/LC09/C02/T1_L2',  # Landsat 9 (2021+)
        'LANDSAT/LC08/C02/T1_L2',  # Landsat 8 (2013+)
        'LANDSAT/LE07/C02/T1_L2',  # Landsat 7 (1999+)
        'LANDSAT/LT05/C02/T1_L2'   # Landsat 5 (1984-2012)
    ]
    
    # Band mapping for different Landsat missions
    BAND_INFO = {
        'LANDSAT_8': {'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10'},
        'LANDSAT_9': {'NIR': 'SR_B5', 'RED': 'SR_B4', 'THERMAL': 'ST_B10'},
        'LANDSAT_7': {'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6'},
        'LANDSAT_5': {'NIR': 'SR_B4', 'RED': 'SR_B3', 'THERMAL': 'ST_B6'}
    }
    
    # Scaling factors
    OPTICAL_SCALE = 0.0000275
    OPTICAL_OFFSET = -0.2
    THERMAL_SCALE = 0.00341802
    THERMAL_OFFSET = 149.0
    
    # LST Calculation Constants
    LST_WAVELENGTH_CONSTANT = 0.00115
    LST_PLANCK_CONSTANT = 1.4388
    
    # Processing parameters
    MAX_PIXELS = 1e9
    REDUCE_SCALE = 1000
    BATCH_SIZE = 5 # Increased for efficiency
    CLOUD_COVER_LIMIT = 70 # Increased from 50 to be more lenient
    
    # Validation limits (allowing older data but warning users)
    MIN_YEAR = 1985
    MAX_YEAR = datetime.now().year
    
    # Visualization parameters
    VIS_PARAMS = {
        'UTFVI': {
            'min': -0.1, 'max': 0.1,
            'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red'],
            'label': 'Urban Thermal Field Variance Index'
        },
        'UHS': {
            'min': 0, 'max': 1,
            'palette': ['lightgray', 'red'],
            'label': 'Urban Hotspots (Binary)'
        },
        'LST': {
            'min': 15, 'max': 50,
            'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red'],
            'label': 'Land Surface Temperature (°C)'
        }
    }

# -----------------------------------------------------------------------------
# 2. INITIALIZATION
# -----------------------------------------------------------------------------
def safe_ee_initialize():
    """Safely initialize Earth Engine"""
    try:
        ee.Initialize()
        return True, "Earth Engine initialized successfully"
    except Exception:
        try:
            ee.Authenticate()
            ee.Initialize()
            return True, "Earth Engine authenticated and initialized"
        except Exception as e:
            return False, f"Failed to initialize Earth Engine: {str(e)}"

print("Initializing Earth Engine...")
init_success, init_message = safe_ee_initialize()
print(init_message)
if not init_success:
    raise Exception("Cannot proceed without Earth Engine initialization")

# -----------------------------------------------------------------------------
# 3. LOAD FLORIDA DATA
# -----------------------------------------------------------------------------
def load_florida_data():
    """Load Florida counties with fallbacks"""
    try:
        print("Loading Florida county data...")
        counties = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
        county_count = counties.size().getInfo()
        if county_count == 0:
            raise Exception("No counties found in primary dataset")
        print(f"Found {county_count} Florida counties")
        state = counties.union()
        try:
            county_names = counties.aggregate_array('NAME').getInfo()
            county_names.sort()
            geometry_options = ['Florida'] + county_names
        except Exception:
            print("Warning: Could not fetch county names, using default list")
            geometry_options = ['Florida', 'Miami-Dade', 'Broward', 'Palm Beach', 'Hillsborough', 'Orange']
        return counties, state, geometry_options, None
    except Exception as e:
        print(f"Error loading county data: {e}")
        return None, None, ['Florida'], str(e)

FLORIDA_COUNTIES, FLORIDA_STATE, GEOMETRY_OPTIONS, load_error = load_florida_data()
analysis_cache = {}
debug_info = []

# -----------------------------------------------------------------------------
# 4. ENHANCED PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def debug_log(message):
    global debug_info
    timestamp = datetime.now().strftime("%H:%M:%S")
    debug_msg = f"[{timestamp}] {message}"
    debug_info.append(debug_msg)
    print(debug_msg)

def apply_scale_factors(image):
    optical = image.select('SR_B.*').multiply(Config.OPTICAL_SCALE).add(Config.OPTICAL_OFFSET)
    thermal = image.select('ST_B.*').multiply(Config.THERMAL_SCALE).add(Config.THERMAL_OFFSET)
    return image.addBands(optical, overwrite=True).addBands(thermal, overwrite=True)

def mask_clouds_and_water(image):
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)
    shadow_mask = qa.bitwiseAnd(1 << 4).eq(0)
    water_mask = qa.bitwiseAnd(1 << 7).eq(0)
    mask = cloud_mask.And(shadow_mask).And(water_mask)
    return image.updateMask(mask)

def get_landsat_bands(spacecraft_id):
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(spacecraft_id)
    return ee.Dictionary(ee.Algorithms.If(is_l8_or_l9, Config.BAND_INFO['LANDSAT_8'], Config.BAND_INFO['LANDSAT_7']))

def calculate_indices(image):
    try:
        spacecraft = ee.String(image.get('SPACECRAFT_ID'))
        bands = get_landsat_bands(spacecraft)
        
        nir = image.select(ee.String(bands.get('NIR')))
        red = image.select(ee.String(bands.get('RED')))
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        
        emissivity = ndvi.multiply(0.004).add(0.986)
        thermal = image.select(ee.String(bands.get('THERMAL')))
        
        lst = thermal.expression(
            'TB / (1 + (LST_WAVE * TB / LST_PLANCK) * log(emis))', {
                'TB': thermal, 'emis': emissivity,
                'LST_WAVE': Config.LST_WAVELENGTH_CONSTANT,
                'LST_PLANCK': Config.LST_PLANCK_CONSTANT
            }).subtract(273.15).rename('LST')
            
        return image.addBands([ndvi, lst, emissivity.rename('EMIS')])
    except Exception as e:
        # This function is mapped over a collection, so we can't use .getInfo()
        # We return an image without LST, which will be filtered out later
        return image

def calculate_thermal_metrics(image, geometry):
    """Calculate UTFVI and UHS with robustness for null stats."""
    lst = image.select('LST')
    stats = lst.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry, scale=Config.REDUCE_SCALE, maxPixels=Config.MAX_PIXELS, bestEffort=True
    )
    
    lst_mean = ee.Number(stats.get('LST_mean'))
    
    # Server-side conditional to check if stats are valid
    def add_metrics(img):
        lst_std = ee.Number(stats.get('LST_stdDev'))
        utfvi = lst.subtract(lst_mean).divide(lst_mean.add(1)).rename('UTFVI')
        threshold = lst_mean.add(lst_std)
        uhs = lst.gt(threshold).rename('UHS')
        return img.addBands([utfvi, uhs])
        
    def no_metrics(img):
        debug_log(f"Regional stats for LST were null. Skipping UTFVI/UHS calculation.")
        return img
        
    return ee.Image(ee.Algorithms.If(lst_mean, add_metrics(image), no_metrics(image)))

def create_composite(year, months, geometry):
    try:
        debug_log(f"Creating composite for {year}, months {months[0]}-{months[1]}")
        start_date = ee.Date.fromYMD(year, months[0], 1)
        end_date = ee.Date.fromYMD(year, months[1] + 1, 1).advance(-1, 'day')
        
        # Merge all available collections
        image_pool = ee.ImageCollection([])
        for coll_id in Config.LANDSAT_COLLECTIONS:
            image_pool = image_pool.merge(ee.ImageCollection(coll_id))

        filtered_collection = (image_pool
                               .filterBounds(geometry)
                               .filterDate(start_date, end_date)
                               .filter(ee.Filter.lt('CLOUD_COVER', Config.CLOUD_COVER_LIMIT)))
        
        total_images = filtered_collection.size().getInfo()
        debug_log(f"Total images found: {total_images}")
        if total_images == 0:
            return None, f"No satellite images found for {year} with current cloud cover settings."
            
        processed = (filtered_collection
                     .map(mask_clouds_and_water)
                     .map(apply_scale_factors)
                     .map(calculate_indices)
                     .filter(ee.Filter.notNull(['LST']))) # Ensure LST was calculated
        
        processed_count = processed.size().getInfo()
        debug_log(f"Images after processing: {processed_count}")
        if processed_count == 0:
            return None, f"All images for {year} were obscured by clouds/shadows after masking."
        
        composite = processed.median()
        final_composite = calculate_thermal_metrics(composite, geometry)
        debug_log("Composite creation successful")
        return final_composite, None
    except Exception as e:
        debug_log(f"Composite creation failed for {year}: {e}")
        return None, f"A GEE error occurred while processing {year}: {e}"

# -----------------------------------------------------------------------------
# 5. UI COMPONENTS
# -----------------------------------------------------------------------------
def create_ui():
    header = widgets.HTML("""<div style="background: linear-gradient(90deg, #1e3c72, #2a5298); color: white; padding: 20px; text-align: center; border-radius: 10px; margin-bottom: 20px;">
                           <h2>🌡️ Florida Urban Heat Analysis Dashboard</h2><p>Satellite-based urban heat island analysis for Florida</p></div>""")
    
    index_dropdown = widgets.Dropdown(options=['LST', 'UTFVI', 'UHS'], value='LST', description='Index:')
    geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, value='Florida', description='Area:')
    start_year_input = widgets.IntText(value=2019, description='Start Year:')
    end_year_input = widgets.IntText(value=2024, description='End Year:')
    delta_dropdown = widgets.Dropdown(options=list(range(1, 6)), value=2, description='Delta (yrs):')
    month_slider = widgets.IntRangeSlider(value=[6, 8], min=1, max=12, description='Months:')
    
    run_button = widgets.Button(description="🚀 Run Analysis", button_style='success')
    debug_button = widgets.Button(description="🔍 Show Debug", button_style='info')
    clear_button = widgets.Button(description="🗑️ Clear", button_style='warning')
    
    status_label = widgets.Label(value="Status: Ready")
    progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:')
    
    map_output = widgets.Output(layout={'height': '600px'})
    table_output = widgets.Output()
    debug_output = widgets.Output()
    
    return locals()

ui = create_ui()

# -----------------------------------------------------------------------------
# 6. MAIN ANALYSIS FUNCTION
# -----------------------------------------------------------------------------
def run_analysis(b):
    global debug_info, analysis_cache
    debug_info = []
    for out in [ui['map_output'], ui['table_output'], ui['debug_output']]:
        with out: clear_output(wait=True)
    ui['progress_bar'].value = 0
    ui['status_label'].value = "Status: Starting analysis..."
    
    try:
        index, area, start_year, end_year, delta, months = (
            ui['index_dropdown'].value, ui['geometry_dropdown'].value, ui['start_year_input'].value,
            ui['end_year_input'].value, ui['delta_dropdown'].value, ui['month_slider'].value
        )
        cache_key = (index, area, start_year, end_year, delta, tuple(months))
        debug_log(f"Analysis parameters: {index}, {area}, {start_year}-{end_year}, delta={delta}, months={months}")
        
        if cache_key in analysis_cache:
            debug_log("Found results in cache! Loading...")
            cached_data = analysis_cache[cache_key]
            display_results(cached_data['df'], index, area, cached_data['composites'], cached_data['years'])
            ui['progress_bar'].value = 100
            ui['status_label'].value = f"Status: Complete! (Loaded {len(cached_data['df'])} data points from cache)"
            return

        if start_year > end_year: raise ValueError("Start year must be <= end year")
        if start_year < Config.MIN_YEAR or end_year > Config.MAX_YEAR:
            raise ValueError(f"Years must be between {Config.MIN_YEAR}-{Config.MAX_YEAR}. Note: Data quality varies significantly before 2013.")
        
        years = list(range(start_year, end_year + 1, delta))
        debug_log(f"Processing years: {years}")
        
        geometry = FLORIDA_STATE if area == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', area)).first().geometry()
        
        results, composites, processing_warnings = [], {}, []
        for i, year in enumerate(years):
            ui['status_label'].value = f"Status: Processing {year}..."
            ui['progress_bar'].value = int((i / len(years)) * 80)
            
            composite, error = create_composite(year, months, geometry)
            if error:
                processing_warnings.append(error)
                debug_log(error)
                continue
            composites[year] = composite
            
            # Extract data
            target_collection = FLORIDA_COUNTIES if area == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', area))
            
            # Check if index exists before trying to reduce regions with it
            available_bands = composite.bandNames().getInfo()
            if index not in available_bands:
                warn_msg = f"'{index}' could not be calculated for {year} (likely sparse data). Skipping statistics for this year."
                processing_warnings.append(warn_msg)
                debug_log(warn_msg)
                continue

            stats = composite.select(index).reduceRegions(
                collection=target_collection, reducer=ee.Reducer.mean(), scale=Config.REDUCE_SCALE,
            ).getInfo()

            for feature in stats['features']:
                props = feature['properties']
                county_name = props.get('NAME', area)
                value = props.get('mean')
                if value is not None:
                    results.append({'County': county_name, 'Year': year, index: round(value, 4)})
        
        ui['progress_bar'].value = 90
        ui['status_label'].value = "Status: Generating outputs..."
        
        if not results:
            error_html = f"""<div style="background-color: #f8d7da; padding: 15px; border-radius: 5px;">
                <h3>❌ Analysis Failed: No data was successfully extracted.</h3>
                <p>This usually happens due to extensive cloud cover in the selected area and time period.</p>
                {'<p><b>Specific reasons:</b><ul>' + ''.join(f'<li>{w}</li>' for w in processing_warnings) + '</ul></p>' if processing_warnings else ''}
                <p>💡 <b>Suggestion:</b> Try widening the 'Months' range or selecting a larger 'Area'.</p>
            </div>"""
            with ui['map_output']: display(widgets.HTML(error_html))
            ui['status_label'].value = "Status: Failed. See details in output."
            return

        df = pd.DataFrame(results)
        analysis_cache[cache_key] = {'df': df, 'composites': composites, 'years': years}
        display_results(df, index, area, composites, years)
        ui['progress_bar'].value = 100
        ui['status_label'].value = f"Status: Complete! ({len(results)} data points)"
        
    except Exception as e:
        ui['status_label'].value = f"Status: Error - {str(e)[:50]}..."
        debug_log(f"Analysis failed: {traceback.format_exc()}")
        with ui['map_output']:
            display(widgets.HTML(f"""<div style="background-color: #f8d7da; padding: 15px; border-radius: 5px;">
                <h3>❌ Analysis Error</h3> <p><strong>Error:</strong> {str(e)}</p>
                <p>Click "Show Debug" button to see detailed logs.</p></div>"""))

def display_results(df, index, area, composites, years):
    """Display analysis results"""
    with ui['map_output']:
        m = geemap.Map()
        m.centerObject(FLORIDA_STATE if area == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', area)), 7 if area == 'Florida' else 9)
        
        vis_params = Config.VIS_PARAMS[index]
        for i, year in enumerate(years):
            if year in composites:
                composite = composites[year]
                # Ensure the band exists before adding the layer
                if index in composite.bandNames().getInfo():
                    m.addLayer(composite.select(index), vis_params, f'{index} {year}', shown=(i == len(years) - 1))
        
        m.add_colorbar(vis_params, label=vis_params['label'])
        m.addLayerControl()
        display(m)
        
    with ui['table_output']:
        pivot_df = df.pivot(index='County', columns='Year', values=index)
        display(widgets.HTML(f"<h3>📊 {index} Data Table</h3>"))
        display(pivot_df.style.background_gradient(cmap='RdYlBu_r'))
        
        display(widgets.HTML("<h4>📈 Summary Statistics</h4>"))
        summary = df.groupby('County')[index].agg(['mean', 'std', 'min', 'max']).round(4)
        display(summary)

def show_debug_info(b):
    """Show debug information"""
    with ui['debug_output']:
        clear_output(wait=True)
        if debug_info:
            debug_text = "\n".join(debug_info[-50:])
            display(widgets.HTML(f"<h3>🔍 Debug Log (Last 50)</h3><pre style='background:#f8f9fa;padding:10px;border-radius:5px;max-height:400px;overflow-y:auto;'>{debug_text}</pre>"))
        else:
            display(widgets.HTML("<p>No debug information available</p>"))

def clear_all(b):
    """Clear all outputs and cache"""
    global analysis_cache, debug_info
    analysis_cache, debug_info = {}, []
    for out in [ui['map_output'], ui['table_output'], ui['debug_output']]:
        with out: clear_output(wait=True)
    ui['progress_bar'].value = 0
    ui['status_label'].value = "Status: Cleared"

# -----------------------------------------------------------------------------
# 7. EVENT HANDLERS & LAYOUT
# -----------------------------------------------------------------------------
ui['run_button'].on_click(run_analysis)
ui['debug_button'].on_click(show_debug_info)
ui['clear_button'].on_click(clear_all)

controls_layout = widgets.VBox([
    widgets.HTML("<div style='background: #e3f2fd; padding: 10px; border-radius: 5px; margin: 5px 0;'><b>🎯 Parameters</b></div>"),
    ui['index_dropdown'], ui['geometry_dropdown'],
    widgets.HTML("<div style='background: #f3e5f5; padding: 10px; border-radius: 5px; margin: 5px 0;'><b>📅 Time Range</b></div>"),
    ui['start_year_input'], ui['end_year_input'], ui['delta_dropdown'], ui['month_slider'],
    widgets.HTML("<div style='background: #e8f5e8; padding: 10px; border-radius: 5px; margin: 5px 0;'><b>🎮 Controls</b></div>"),
    widgets.HBox([ui['run_button'], ui['debug_button'], ui['clear_button']]),
    ui['progress_bar'], ui['status_label']
])
output_tabs = widgets.Tab([ui['table_output'], ui['debug_output']], titles=['📊 Data', '🔍 Debug'])
left_panel = widgets.VBox([controls_layout, output_tabs], layout=widgets.Layout(width='35%', padding='10px', display='flex', flex_flow='column'))
right_panel = widgets.VBox([ui['map_output']], layout=widgets.Layout(width='65%'))
full_layout = widgets.VBox([ui['header'], widgets.HBox([left_panel, right_panel])])

# -----------------------------------------------------------------------------
# 9. WELCOME MESSAGE & DISPLAY
# -----------------------------------------------------------------------------
welcome_msg = """
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 20px;">
    <h2>🌡️ Welcome to Florida Urban Heat Analysis!</h2>
    <h3>🚀 Quick Start Guide:</h3>
    <ul>
        <li><strong>Index:</strong> LST (Land Surface Temperature) is the most reliable starting point.</li>
        <li><strong>Area:</strong> Start with 'Florida' for a broad overview.</li>
        <li><strong>Time:</strong> Use the default years for high-quality recent data.</li>
    </ul>
    <p>💡 <em><strong>Note:</strong> Florida summers are very cloudy! For small areas like a single county, you may need to widen the 'Months' range (e.g., April-September) to ensure the analysis can find clear satellite images.</em></p>
</div>
"""
with ui['map_output']:
    display(widgets.HTML(welcome_msg))

print("🚀 Florida Urban Heat Analysis Dashboard - Ready!")
print("📋 Try the recommended settings, then click 'Run Analysis'.")
display(full_layout)

Initializing Earth Engine...
Earth Engine initialized successfully
Loading Florida county data...
Found 67 Florida counties


🚀 Florida Urban Heat Analysis Dashboard - Ready!
📋 Try the recommended settings, then click 'Run Analysis'.
